In [1]:
from urdfenvs.robots.generic_urdf import GenericUrdfReacher
from urdfenvs.sensors.full_sensor import FullSensor
from urdfenvs.urdf_common import UrdfEnv
import os
import numpy as np

pybullet build time: May 20 2022 19:45:31


In [2]:
from object import movable_obstacle

In [3]:
robots = [
    GenericUrdfReacher(urdf="panda_with_gripper.urdf", mode="tor"),
]

render: bool = True

roboticEnv = UrdfEnv(render=render, robots=robots)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Microsoft Corporation
GL_RENDERER=D3D12 (NVIDIA GeForce RTX 3050 Ti Laptop GPU)
GL_VERSION=3.3 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=3.30
pthread_getconcurrency()=0
Version = 3.3 (Core Profile) Mesa 21.2.6
Vendor = Microsoft Corporation
Renderer = D3D12 (NVIDIA GeForce RTX 3050 Ti Laptop GPU)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Microsoft Corporation
ven = Microsoft Corporation


GL error 0x500 detected in glGenBuffers


In [5]:
from reward import DistanceReward
from forwardkinematics.urdfFks.pandaFk import PandaFk

sensor = FullSensor(['position'], ['position', 'size'], variance=0.0)
roboticEnv.add_sensor(sensor, [0])
roboticEnv.add_obstacle(movable_obstacle)

fk_panda = PandaFk()
roboticEnv.set_reward_calculator(DistanceReward(fk_panda=fk_panda))

roboticEnv.set_spaces()

ob, *_ = roboticEnv.reset()

Unknown tag "inertial" in /robot[@name='panda']
Unknown tag "inertial" in /robot[@name='panda']
/home/behradx/anaconda3/envs/SB3/lib/python3.9/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float64
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Link with name world does not has a parent. Link name is skipped.
Link with name world does not has a parent. Link name is skipped.


In [ ]:
# import numpy as np

# env.reset()
# while True:
#     action = env.action_space.sample()
#     action = np.array([ 0, 0 , 0,  -1,  0,
#        0, 0 , 0, 0], dtype=np.float32)
#     obs, reward, done, truncated, info = env.step(action)


In [ ]:
# Stable baselines 3
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise

from gymnasium.wrappers import FlattenObservation


In [ ]:
MODEL_NAME = 'TD3-001'
MODEL_CLASS = TD3

In [ ]:
models_dir = 'models/' + MODEL_NAME
logdir = 'tb_logs'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)

In [ ]:
roboticEnv = FlattenObservation(roboticEnv)

In [ ]:
from urdfenvs.wrappers.sb3_float32_action_wrapper import SB3Float32ActionWrapper

roboticEnv = SB3Float32ActionWrapper(roboticEnv)

In [ ]:
from normalize_action import MapActionWrapper

roboticEnv = MapActionWrapper(roboticEnv)

In [ ]:
from gymnasium.wrappers import TimeLimit

roboticEnv = TimeLimit(env=roboticEnv, max_episode_steps= 50)

In [ ]:
roboticEnv.action_space.is_bounded

<bound method Box.is_bounded of Box(-1.0, 1.0, (9,), float32)>

In [ ]:
env.action_space

Box(-1.0, 1.0, (4, 9), float32)

In [ ]:
# from stable_baselines3.common.env_checker import check_env

# check_env(env, warn=True)

In [ ]:
import numpy as np
from learning_rate_schedule import linear_schedule
# n_actions = env.action_space.shape[-1]
# action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = TD3("MlpPolicy", 
                    roboticEnv, 
                    verbose=0, 
                    tensorboard_log= logdir, 
                    # learning_rate=linear_schedule(1),
                    batch_size= 256, # It's better to be high. Default is 256. 
                    # buffer_size= 100,
                    # gamma= 0.99, # Discount Factor 
                    # action_noise=action_noise, 
                    device='cpu',
                    
                    )

In [ ]:
TIMESTEPS = 1_000_000

model.learn(total_timesteps=TIMESTEPS, log_interval=10, tb_log_name=MODEL_NAME,
             progress_bar=True, 
               )
model.save(f"{MODEL_NAME}-model")

Output()

Unknown tag "inertial" in /robot[@name='panda']

Unknown tag "inertial" in /robot[@name='panda']

Link with name world does not has a parent. Link name is skipped.

Link with name world does not has a parent. Link name is skipped.

...

Link with name world does not has a parent. Link name is skipped.

🎁 Reward: 15.58048202911801

TypeError: only size-1 arrays can be converted to Python scalars

X connection to 172.17.112.1:0 broken (explicit kill or server shutdown).


: 

In [ ]:
# obs = env.reset()
# model.predict(obs)

In [ ]:
obs = roboticEnv.reset()
for _ in range(10):
    action, states = model.predict(obs, deterministic=True)
    obs, reward, done, info = roboticEnv.step(action)
    roboticEnv.render()
    if done:
      obs = roboticEnv.reset()

/home/behradx/anaconda3/envs/SB3/lib/python3.9/site-packages/stable_baselines3/common/policies.py:260: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  observation = np.array(observation)


ValueError: Error: Unexpected observation shape (2,) for Box environment, please use (24,) or (n_env, 24) for the observation shape.

In [ ]:
# env.close()